<a href="https://colab.research.google.com/github/jayantc14/Yelp_Data_Analysis/blob/main/BERT_final_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [2]:
!pip install tokenization

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 11.5 MB/s 
     |████████████████████████████████| 596 kB 42.0 MB/s 
     |████████████████████████████████| 61 kB 447 kB/s 
     |████████████████████████████████| 895 kB 33.9 MB/s 
     |████████████████████████████████| 3.3 MB 34.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
% pip install sentencepiece

     |████████████████████████████████| 1.2 MB 11.1 MB/s 


In [6]:
import tensorflow_hub as hub
import tokenization
module_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [7]:
import pandas as pd

In [8]:
df = pd.read_csv("/content/drive/MyDrive/yelp_dataset/final_data_yelp2.csv")


In [9]:
df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'business_id', 'city', 'state_x',
       'user_id', 'review_stars', 'useful_x', 'date', 'useful_y',
       'fans', 'Smogscore', 'review_word_count', 'Polarity',
       'Analysis_TextBlob', 'Vader Sentiment', 'Vader Analysis', 'fips',
       'cases', 'deaths'],axis = 1,inplace = True)

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
#y = df['review_stars']
#X = df.drop("DEATH_EVENT", axis = 1)
X = df.drop(["label"], axis = 1)
train, test = train_test_split(df, test_size=0.2, random_state=25)

In [12]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence) + [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [13]:
def build_model(bert_layer, max_len=512):
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    net = tf.keras.layers.Dense(64, activation='relu')(clf_output)
    net = tf.keras.layers.Dropout(0.2)(net)
    net = tf.keras.layers.Dense(32, activation='relu')(net)
    net = tf.keras.layers.Dropout(0.2)(net)
    out = tf.keras.layers.Dense(2, activation='softmax')(net)
    
    model = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [14]:
import numpy as np

In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Conv1D, MaxPool1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [16]:
import keras

In [17]:
import tensorflow as tf

In [18]:
from tensorflow.keras.utils import to_categorical

In [19]:
!pip install keras.utils

  Created wheel for keras.utils: filename=keras_utils-1.0.13-py3-none-any.whl size=2656 sha256=eb86cedbfce691d9d66b07a6452f1984b1514082bd80d26c8670b2ee124176b4
  Stored in directory: /root/.cache/pip/wheels/d0/dd/3b/493952a5240d486a83805d65360dedadbadeae71d25e2c877f
Successfully built keras.utils


In [20]:
#from keras.utils import to_categorical

In [21]:
train

,text,label
63082,So Im hoping this was just an off night for yo...,1.0
139977,The first time I visited AF was during a cold ...,1.0
67546,Excellent food authentic and tasty If you love...,1.0
146484,Thank you Katie My amazing bartender Katie O w...,1.0
1769,This is definitely one of the best authentic b...,1.0
...,...,...
130365,Review for brunch The vibe is very cool and hi...,1.0
92303,In these trying times there isnt a day that go...,1.0
6618,First timer This place was so good Its not ver...,1.0
90430,I generally only like thin crust pizza but thi...,1.0


In [22]:
max_len = 150
train_input = bert_encode(train.text.values, tokenizer, max_len=max_len)
test_input = bert_encode(test.text.values, tokenizer, max_len=max_len)


In [23]:
train.reset_index(drop=True,inplace = True)

In [24]:
train

,text,label
0,So Im hoping this was just an off night for yo...,1.0
1,The first time I visited AF was during a cold ...,1.0
2,Excellent food authentic and tasty If you love...,1.0
3,Thank you Katie My amazing bartender Katie O w...,1.0
4,This is definitely one of the best authentic b...,1.0
...,...,...
158354,Review for brunch The vibe is very cool and hi...,1.0
158355,In these trying times there isnt a day that go...,1.0
158356,First timer This place was so good Its not ver...,1.0
158357,I generally only like thin crust pizza but thi...,1.0


In [25]:
train_labels = tf.keras.utils.to_categorical(train.label.values, num_classes=None)

In [26]:
train_labels

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [27]:
model = build_model(bert_layer, max_len=max_len)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 150)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 150)]        0           []                               
                                                                                                  
 segment_ids (InputLayer)       [(None, 150)]        0           []                               
                                                                                                  
 keras_layer (KerasLayer)       [(None, 768),        109482241   ['input_word_ids[0][0]',         
                                 (None, 150, 768)]                'input_mask[0][0]',         

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [28]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)

train_history = model.fit(
    train_input, train_labels, 
    validation_split=0.2,
    epochs=1,
    callbacks=[checkpoint, earlystopping],
    batch_size=32,
    verbose=1)

3959/3959 [==============================] - ETA: 0s - loss: 0.1236 - accuracy: 0.9485
Epoch 00001: val_accuracy improved from -inf to 0.96461, saving model to model.h5
3959/3959 [==============================] - 8083s 2s/step - loss: 0.1236 - accuracy: 0.9485 - val_loss: 0.0848 - val_accuracy: 0.9646


In [29]:
model.load_weights('model.h5')
y_pred = model.predict(test_input)

In [30]:
y_pred

array([[1.4567558e-03, 9.9854326e-01],
       [5.1978528e-05, 9.9994802e-01],
       [3.5661332e-02, 9.6433872e-01],
       ...,
       [2.4052409e-02, 9.7594756e-01],
       [6.4965396e-04, 9.9935037e-01],
       [1.1252201e-06, 9.9999893e-01]], dtype=float32)

In [31]:
test

,text,label
23146,Voodoo doughnut was cool It wasnt the doughnut...,1.0
27022,Simply a solid soup or sandwich place sit dow...,1.0
163469,Came here for lunch quick and tasty Pricier si...,1.0
11507,Another underwhelming Mexican spot that has po...,1.0
67940,I went to this place about a year ago on a whi...,1.0
...,...,...
38537,On the day I visited they didnt have any vegan...,1.0
6624,Always read good reviews of this establishment...,1.0
144662,I dont know why I got the prime rib haaaa I ne...,1.0
55348,Not what I had in mind at all for Mediterranea...,1.0


In [36]:
y_hat = y_pred.argmax(axis = 1)[:,None]

In [37]:
y_hat

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [38]:
y_hat2 = y_hat.flatten()

In [39]:
y_hat2

array([1, 1, 1, ..., 1, 1, 1])

In [40]:
test.reset_index(drop=True,inplace = True)

In [41]:
y_test = test['label']

In [42]:
y_test = y_test.to_numpy()

In [43]:
y_test

array([1., 1., 1., ..., 1., 1., 1.])

In [44]:
accuracy_score(y_test, y_hat2)

0.9640313210406668

In [45]:
from sklearn import metrics

In [46]:
print(metrics.classification_report(y_test, y_hat2))

              precision    recall  f1-score   support

         0.0       0.82      0.81      0.81      3883
         1.0       0.98      0.98      0.98     35707

    accuracy                           0.96     39590
   macro avg       0.90      0.89      0.90     39590
weighted avg       0.96      0.96      0.96     39590

